# Redes Neurais Artificiais com PyTorch
# Sumário

[1. Operações com Tensores](#heading--1)

[2. DataSets e DataLoaders](#heading--2)

   * [2.1 Carregando um DataSet](#heading--2-1)
    
   * [2.2 Criando um DataSet](#heading--2-2)
    
   * [2.3 Preparando os Dados para Treino com DataLoaders](#heading--2-3)
   
[3. Construindo uma Rede Neural](#heading--3)

   * [3.1 Definindo a Classe](#heading--3-1)
   * [3.2 Camadas e Parâmetros](#heading--3-2)
   
[4. Diferenciação Automática com __torch.autograd__](#heading--4)

   * [4.1 Computando Gradientes](#heading--4-1)
   * [4.1 Desabilitandi o Rastreamento dos gradientes](#heading--4-2)

[5. Treninando e Testando o Modelo](#heading--5)

   * [5.1 Hiperparâmetros](#heading--5-1)
   * [5.1 Funções de Perda e Otimizador](#heading--5-2)
   * [5.1 Implementação](#heading--5-3)
   * [5.1 Salvando e Carregando um Modelo](#heading--5-4)

<a id="heading--1"></a>
## Operações com Tensores
Os tensores no PyTorch são matrizes multidimensionais que representam dados. Eles são semelhantes aos arrays do Numpy, mas têm algumas vantagens, como poder ser operados em GPUs e armazenar o histórico de cálculos para facilitar a diferenciação automática.

Os tensores podem ser criados a partir de listas Python, arrays Numpy ou valores aleatórios usando a classe torch.Tensor ou suas subclasses2. Os tensores são a base para o aprendizado profundo com PyTorch, pois permitem a construção e o treinamento de redes neurais complexas.

In [1]:
import matplotlib.pyplot as plt
import torch

import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

In [2]:
tensor = torch.rand(3,4)
tensor1 = torch.ones(2,2)

print("Tensor aleatório:\n", tensor)
print("\nTensor preenchido com 1s:\n", tensor1)
print("\n")

#Indexação e fatiamento semelhante a Numpy

print("Indexação e Fatiamento:\n")
tensor = torch.ones(4, 4)
print(f"Primeira Linha: {tensor[0]}")
print(f"Primeira Coluna: {tensor[:, 0]}")
print(f"Última Coluna: {tensor[..., -1]}")
tensor[:,1] = 0
print(tensor)

print("Concatenação:\n")
t1 = torch.cat([tensor, tensor, tensor], dim=0)    #dimensão do novo vetor
print(t1)

Tensor aleatório:
 tensor([[0.9095, 0.4715, 0.8731, 0.2251],
        [0.6308, 0.3806, 0.0483, 0.8550],
        [0.9168, 0.1286, 0.8631, 0.9469]])

Tensor preenchido com 1s:
 tensor([[1., 1.],
        [1., 1.]])


Indexação e Fatiamento:

Primeira Linha: tensor([1., 1., 1., 1.])
Primeira Coluna: tensor([1., 1., 1., 1.])
Última Coluna: tensor([1., 1., 1., 1.])
tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]])
Concatenação:

tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]])


In [3]:
#Mutiplicação Matricial
y0 = tensor.T

y1 = tensor @ tensor.T
y2 = tensor.matmul(tensor.T)
print("\nMatriz Transposta:\n", y0)
print("\nMutiplicação Matricial:\n", y2)


Matriz Transposta:
 tensor([[1., 1., 1., 1.],
        [0., 0., 0., 0.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])

Mutiplicação Matricial:
 tensor([[3., 3., 3., 3.],
        [3., 3., 3., 3.],
        [3., 3., 3., 3.],
        [3., 3., 3., 3.]])


In [4]:
#Multiplicação termo a termo
z1 = tensor * tensor
z2 = tensor.mul(tensor)
print("Multiplicação termo a termo:\n", z2)

Multiplicação termo a termo:
 tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]])


<a id="heading--2"></a>
## DataSets e DataLoaders

O código para processar amostras de dados pode ficar confuso e difícil de manter; idealmente, queremos que nosso código de conjunto de dados seja desacoplado do nosso código de treinamento de modelo para melhor legibilidade e modularidade. 

*Dataset* armazena as amostras e seus rótulos correspondentes, e *DataLoader* envolve um iterável em torno do *Dataset* para permitir um fácil acesso às amostras. PyTorch fornece duas primitivas de dados: **torch.utils.data.DataLoader** e __torch.utils.data.Dataset__ que permitem que você use conjuntos de dados pré-carregados, bem como seus próprios dados

<a id="heading--2-1"></a>
### Carregando um DataSet

Aqui está um exemplo de como carregar o conjunto de dados Fashion-MNIST do TorchVision1. Fashion-MNIST é um conjunto de dados de imagens de artigos da Zalando, consistindo de 60.000 exemplos de treinamento e 10.000 exemplos de teste2. Cada exemplo é composto por uma imagem em escala de cinza de 28×28 e um rótulo associado de uma das 10 classes.

Carregamos o conjunto de dados FashionMNIST com os seguintes parâmetros: 

**root** é o caminho onde os dados de treino/teste são armazenados,

**train** especifica o conjunto de dados de treinamento ou teste,

**download=True** baixa os dados da internet se eles não estiverem disponíveis em root.

**transform** e **target_transform** especificam as transformações de características e rótulos

In [5]:
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor


training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

<a id="heading--2-2"></a>
### Criando um Dataset
Para a riação de um objeto que herde a classe Dataset, é necessário a utilização de três funções:

__init__: Fornece o conjunto de dados para a calsse;
__len__: Retorna o tamanho do conjunto de dados;
__getitem__: Retorna o dado de indíce especificado.

Os dados podem ser armazenados em arquivos externos como arquivos CSV, que contém tanto o conjunto de dados como a etiqueta dos dados. Também é possível passar parâmetros que representem funções, para geração de dados na própria criação da classe.


In [6]:
from torch.utils.data import Dataset

from torch.distributions import Normal
class Dados(Dataset):
    def __init__(self, media, desvio, qamostras):
        # Cria uma distribuição normal com média 0 e desvio padrão 1
        self.distribuicao = Normal(torch.tensor([0.0]), torch.tensor([1.0]))

        # Gera dez amostras aleatórias
        self.amostras = self.distribuicao.sample((qamostras,))
        
        self. alvos = torch.rand(self.amostras.shape)
    
    def __len__(self):
        return len(self.amostras)
    
    def __getitem__(self, i):
        return self.amostras[i], self.alvos[i]



<a id="heading--2-3"></a>
### Preparando os Dados para Treino com DataLoaders

É interesssante que para o treinamento do modelo, os dados sejam passados em lotes, e a cada _época_, eles sejam reorganizados, para evitar o overfitting - Situação indesejável na qual o modelo se adequa perfeitamente aos dados, mas não consegue generalizar para outros conjuntos de dados, tornando o modelo inútil -, e usar o multiprocessamento do Python para acelerar a recuperação de dados.

DataLoader é um iterável que faz justamente esse processo, em uma API(Interface de Programação de Aplicações) fácil de usar

In [7]:
from torch.utils.data import DataLoader

#Define um conjunto de dados a partir da classe Dados para treinamento e teste
training_dataset = Dados(0, 1, 1000)
test_dataset = Dados(0,1,100)

#Define os dataloaders para treinamento e teste
train_dataloader = DataLoader(training_dataset, batch_size = 200, shuffle = True)
test_dataloader = DataLoader(test_dataset, batch_size = 20, shuffle = True)


# Cria um iterador para o DataLoader
data_iter = iter(train_dataloader)

# Obtém o primeiro lote de dados
first_batch_samples, first_batch_targets = next(data_iter)

# Imprime o formato do primeiro lote de dados
print(first_batch_samples.shape)
print(first_batch_targets.shape)

for i, data in enumerate(train_dataloader):
    print(f"Batch: {i}, Data: {data}")
    if i > 10:  # Limitar a quantidade de saída
        break

torch.Size([200, 1])
torch.Size([200, 1])
Batch: 0, Data: [tensor([[-0.5707],
        [ 0.2190],
        [-1.3237],
        [ 1.6763],
        [ 0.5365],
        [-0.1446],
        [ 0.2325],
        [ 0.8397],
        [-1.2482],
        [ 0.1841],
        [ 0.9017],
        [-0.6452],
        [-0.2536],
        [ 0.3142],
        [ 0.6364],
        [ 1.0160],
        [ 0.6792],
        [-1.1867],
        [-0.0431],
        [ 0.2760],
        [-0.3222],
        [-0.1634],
        [ 0.3855],
        [ 1.5882],
        [-0.7877],
        [-1.4374],
        [ 0.4654],
        [ 1.0757],
        [-0.7070],
        [-3.1457],
        [-2.0231],
        [ 0.3934],
        [-1.8541],
        [ 0.4614],
        [ 0.2316],
        [-0.8783],
        [ 0.4064],
        [-0.6521],
        [-0.7988],
        [ 0.3559],
        [ 0.4037],
        [ 1.0780],
        [ 2.2621],
        [ 0.9758],
        [-0.6729],
        [ 0.6896],
        [-0.2085],
        [-0.5761],
        [-0.8101],
        [ 

<a id="heading--3"></a>
## Construindo uma Rede Neural

As redes neurais são modelos computacionais, formadas por camadas de neurônios artificiais, estruturas que realizam operações paramétricas com os dados de entrada. O namespace __torch.nn__ fornece todos os blocos necessários para a construção de uma rede neural.

Todo módulo no PyTorch é uma subclasse do __nn.Module__, e a rede é basicamente um módulo composto por outros módulos. Aqui, será desenvolvido um modelo de regressão para aproximação de valores aleatórios, com distribuição de probabilidade Gaussiana.


<a id="heading--3-1"></a>
### Definindo a Classe

Nós definimos as camadas da nossa rede neural na função **_ _ init _ _**, e toda subclasse do módulo implementa as operações de entrada de dados no método forward.


In [8]:
from torch import nn

class NN(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
        nn.Linear(1, 10),
        nn.ReLU(),
        nn.Linear(10,5),
        nn.ReLU(),
        nn.Linear(5,1),
        )
    
    def forward(self, x):
        y = self.linear_relu_stack(x)
        return y

<a id="heading--3-2"></a>
### Camadas e Parâmetros

As camadas da rede neural são aplicadas no método init, geralmente como parâmetros para a função Sequential, que executa as camadas na ordem em que são dadas.
O módulo fornece, além de funções lineares, e diversas funções de ativação, como a **ReLU**, que representão não linearidades na nossa rede neural. Essas funções de ativação são necessárias para a aproximação de sistemas não lineares.

Essas camadas possuem parâmetros que são utilizados para a aproximação de funções a partir dos dados de entrada, pesos são multiplicados e biases são somados a eles. Esses parâmetros podem ser acessados usando os métodos parameters() ou named_parameters() do modelo.

In [9]:
X = torch.rand(10, 1)
model = NN()
Y = model(X)
print(f"Estrutura do modelo: {Y}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Estrutura do modelo: tensor([[0.2358],
        [0.2169],
        [0.2054],
        [0.2115],
        [0.2117],
        [0.2115],
        [0.2085],
        [0.2185],
        [0.2058],
        [0.2286]], grad_fn=<AddmmBackward0>)


Layer: linear_relu_stack.0.weight | Size: torch.Size([10, 1]) | Values : tensor([[-0.5531],
        [-0.6762]], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([10]) | Values : tensor([-0.3908, -0.7253], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([5, 10]) | Values : tensor([[ 0.2604, -0.2064, -0.2074,  0.2656,  0.2973,  0.2744,  0.0609,  0.0482,
          0.1465,  0.0844],
        [-0.0296,  0.1473,  0.0841, -0.0370, -0.1730, -0.0518,  0.0763, -0.0516,
          0.1126, -0.2061]], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.bias | Size: torch.Size([5]) | Values : tensor([-0.2728, -0.1454], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.4.weight | Size: torch.Size([1, 5]) | Va

<a id="heading--4"></a>
## Diferenciação Automática com __torch.autograd__

O algoritmo mais utlizado no treinamento de redes neurais é a retro-propagação, na qual os parâmetros associados à entrada são  ajustados a partir do gradiente da função de perda associada a cada parâmetro. 
Para computar os gradientes de cada variável nos ativamos sua propriedade *requires_grad para True*

O código abaixo aplica uma camada linear à entrada, sendo w o peso associado aos dados e b o bias somado ao resultado. A função de perda é dada por loss.

In [10]:
x = torch.ones(5)  # tensor de entrada
y = torch.zeros(3)  # saída desejada
w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x, w)+b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

<a id="heading--4-1"></a>
### Computando Gradientes

O método loss.backwards() calcula as derivadas parcias da perda com relação a **w** e a **b**, os nossos parâmetros, aplicados para um **x** e **y** constantes. Nós então podemos obter esses gradietes a partir da função *tensor.grad*

In [11]:
loss.backward()
print(w.grad)
print(b.grad)

tensor([[0.0055, 0.0161, 0.1021],
        [0.0055, 0.0161, 0.1021],
        [0.0055, 0.0161, 0.1021],
        [0.0055, 0.0161, 0.1021],
        [0.0055, 0.0161, 0.1021]])
tensor([0.0055, 0.0161, 0.1021])


<a id="heading--4-2"></a>
### Desabilitando o Rastreamento dos Gradientes

Por padrão, todos os tensores já vem com o seu requires_grad = True, e rastreiam o seu histórico computacional e suporte a computação dos gradientes. Em certas situações, não é desejável que os gradientes sejam associados aos parâmetros, há somente a necessidade de testar o método forward da rede. 

É possível desativar o requires_grad de um tensor de agumas formas:

In [12]:
#Utilizando o bloco torch.no_grad()
with torch.no_grad():
    z = torch.matmul(x, w)+b

print(z.requires_grad)

#Utilizando o método detach():
z = torch.matmul(x, w)+b
print("\nAntes do detach: ", z.requires_grad)
z_det = z.detach()
print("\nDepois do detach: ", z_det.requires_grad)

False

Antes do detach:  True

Depois do detach:  False


<a id="heading--5"></a>
## Treinando e Testando o Modelo

Agora que já temos um modelo de rede neural, o próximo passo é treinar, avaliar e testar o nosso modelo otimizando os parâmetros dos nossos dados. No processo iterativo de treinamento, cada iteração atribui parâmetros iniciais aos dados, calucla a função de erro, e ajusta os parâmetros iniciais a partir da derivada da função de perda em relação aos parâmetros

<a id="heading--5-1"></a>
### HiperParâmetros

O processo de otimização pode ser controlado utilizando o que chamamos de hiperparâmetros, que podem impacctar no treinamento do modelo e na taxa de convergência. Os hiperparâmetros que são utilizáveis são:

__Número de épocas__:  A quantidade de vezes para iterar sobre os dados;

__Tamanho do lote__: O número de amostras que serão passadas ao modelo antes da atualização dos parâmetros;

**Taxa de aprendizado**: O quanto atualizar os parâmetros a cada lote ou época. Valores pequenos podem gerar um processo muito lento, mas valores altos podem  causar comportamentos imprevisíveis.

Cada época consiste de duas principais partes:

*Loop de treino*: itera sobre o dataset de treinamento e tenta alcançar os parâmetros ótimos;

*Loop de teste*: Itera sobre o dataset de teste para avaliar o aprendizado do modelo

In [13]:
learn_rate = 1e-2
batch = len(test_dataloader)
epochs = 10

<a id="heading--5-2"></a>
### Funções de Perda e Otimizador

As funções de perda são responsáveis por medir  o grau de erro entre os resultados obtidos pelo modelo e o conjunto de dados alvo. É justamente essa função que desejamos minimizar durante o treinamento.

O torch disponibiliza diferentes tipos de funções de perda, para diferenes problemas: *nn.MSELoss*, para problemas de regressão, *nn.NLLLoss*, para problemas de classificação, *nn.CrossEntropyLoss*, que vai combinar a função de classificação com o método de probabilidade *nn.LogSoftMax*

Para nosso modelo de regressão, utilizaremos a MSELoss.

Os algoritmos de otimização definem como o processo de ajuste dos parâmetros será feito. existem diversos orimizadores disponíveis na biblioteca do Pytorch. No nosso exemplo, utilizaremos o otimizador Adam, que combina os processos SGD(*Stochastic Gradient Descent*) e RMSProp().

Nós inicializamos o otimizador ao registrar os parâmetros  que precisam ser treinados, e passando a taxa de aprendizado como hiperparâmetro.

In [14]:
loss = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr =  learn_rate)

Dentro do loop de treino, a otimização ocorre em três etapas:

* Chamada do método optimizer.zero_grad() para resetar os gradientes. Por padrão, esses gradientes são somados a cada iteração como isso não é desejável em nosso caso, nós os setamos para 0 a cada iteração;
 
 
* Retropropagamos a função de perda com loss.backward(), que deposita os gradientes nos respectivos parâmetros;


* Chamada do optimizer.step(), para ajustar os parâmetros com os gradientes obtidos no passo anterior.

<a id="heading--5-3"></a>
### Implementação

Aqui, definimos *train_loop*, que reitera sobre nosso código de otimização, e *test_loop*, que avalia o nosso modelo a partir dos nossos dados de teste.

In [15]:
def train_loop(dataloader, model, loss, optimizer):
    size = len(dataloader.dataset)
    
    model.train()
    for batch, (x, y) in enumerate(dataloader):
        predict = model(x)
        loss_value = loss(predict, y)
        
        loss_value.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        if batch % 1 == 0:
            loss_value, current = loss_value.item(), (batch+1)*len(x)
            print(f"loss: {loss_value:>7f}[{current:>5d}/{size:>5d}]" )


def test_loop(dataloader, model, loss):
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss =0
    
    with torch.no_grad():
        for x, y in dataloader:
            predict = model(x)
            test_loss += loss(predict, y).item()
    test_loss /= num_batches
    print(f"Perda média: {test_loss:>8f} \n")

In [16]:
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss, optimizer)
    test_loop(test_dataloader, model, loss)

print("Done!")

Epoch 1
-------------------------------
loss: 0.167467[  200/ 1000]
loss: 0.177077[  400/ 1000]
loss: 0.125214[  600/ 1000]
loss: 0.126209[  800/ 1000]
loss: 0.120535[ 1000/ 1000]
Perda média: 0.112350 

Epoch 2
-------------------------------
loss: 0.097789[  200/ 1000]
loss: 0.105444[  400/ 1000]
loss: 0.084260[  600/ 1000]
loss: 0.096649[  800/ 1000]
loss: 0.101720[ 1000/ 1000]
Perda média: 0.104148 

Epoch 3
-------------------------------
loss: 0.085985[  200/ 1000]
loss: 0.090928[  400/ 1000]
loss: 0.087269[  600/ 1000]
loss: 0.086458[  800/ 1000]
loss: 0.092727[ 1000/ 1000]
Perda média: 0.100709 

Epoch 4
-------------------------------
loss: 0.089251[  200/ 1000]
loss: 0.086766[  400/ 1000]
loss: 0.078078[  600/ 1000]
loss: 0.084422[  800/ 1000]
loss: 0.083376[ 1000/ 1000]
Perda média: 0.095413 

Epoch 5
-------------------------------
loss: 0.088626[  200/ 1000]
loss: 0.084103[  400/ 1000]
loss: 0.082385[  600/ 1000]
loss: 0.078797[  800/ 1000]
loss: 0.075734[ 1000/ 1000]
Perd

Aqui nós podemos observar dois comportamentos interessantes na perda:

* Ela aparenta decrescer em média de forma lenta. Isso se deve muito provavelmente de que estamos tratando de um problema de regressão, com valores aleatórios, e é extremamente complicado que a rede neural consiga aprender a resolver esse problema.

* O outro comportamento, se refere às oscilações da perda média, observe que ela decresce e cresce em alguns momentos. Isso provavelmente se deve ao nosso learning_rate, que deve estar alto demais, e causando essass oscilações.

<a id="heading--5-4"></a>
### Salvando e Carregando Modelos

É possível salvar os parâmetros aprendidos pelo modelo em um dicionário de estado state_dictonary, extermnamente, e podem ser carregados para um anova instância do mesmo modelo:

Além disso, pode-se salvar o próprio modelo, considerando sua estrutura, alés dos pesos que foram calculados.

In [17]:
#Salva os parâmetros do modelo
torch.save(model.state_dict(), 'model_weights.pth')

#Salva a aestrutura do modelo
torch.save(model, 'model.pth')

In [18]:
#Carregando os parâmetros do modelo
model2 = NN()
model.load_state_dict(torch.load('model_weights.pth'))
model.eval()

#Carregando o modelo em si
model3 = torch.load('model.pth')